In [1]:
import pandas as pd
import datetime
import re

data = pd.read_csv("data_bll.csv", sep=";")  # Load
data["time"] = pd.to_datetime(data["time"], format="%d.%m.%Y-%H:%M:%S")  # Make time to type time

vent = pd.read_csv("vent.csv", sep=";")
vent["time"] = pd.to_datetime(vent["time"], format="%Y-%m-%d %H:%M:%S")  # Make time to type time

dush = pd.read_csv("dush.csv", sep=";")
dush["time"] = pd.to_datetime(dush["time"], format="%Y-%m-%d %H:%M:%S")  # Make time to type time

In [2]:
to_process = pd.read_csv("data.../lüften_oben.csv")
shower_max = max(vent["Id"])
to_process

,Monat,Tag,Stunde,Beginn,Stunde Ende,Ende,Config
0,3,18,7,30,7,43,1
1,3,19,16,23,16,36,1
2,3,22,20,58,21,5,1
3,3,25,5,55,6,20,1
4,3,27,6,5,6,25,1
5,4,1,6,28,6,43,1
6,4,3,13,40,13,48,1
7,4,4,22,27,22,33,1
8,4,22,21,25,21,40,1


In [3]:
# for shower
showering = []

current_ID = shower_max +1

for i in range(len(to_process)):
    year, month = 2022, to_process["Monat"][i]
    day_start, day_end = to_process["Tag"][i], to_process["Tag"][i]
    if to_process["Minuten"][i] > to_process["Minute Ende"][i]:
        time_end = to_process["Stunde"][i]+1
    else:
        time_end = to_process["Stunde"][i]
    start = datetime.datetime(year, month, day_start, to_process["Stunde"][i], to_process["Minuten"][i])
    end = datetime.datetime(year, month, day_end, time_end, to_process["Minute Ende"][i])
    time_mask = (data['time'] >= start) & \
            (data['time'] <= end)
    showering.append(data[time_mask].copy())
    showering[-1]["Id"] = current_ID
    current_ID += 1

showering = pd.concat(showering, ignore_index=True)

In [4]:
# for venting
vent_now = []

current_ID = shower_max +1

for i in range(len(to_process)):
    year, month = 2022, to_process["Monat"][i]
    day_start, day_end = to_process["Tag"][i], to_process["Tag"][i]
    
    start = datetime.datetime(year, month, day_start, to_process["Stunde"][i], to_process["Beginn"][i])
    end = datetime.datetime(year, month, day_end, to_process["Stunde Ende"][i], to_process["Ende"][i])
    time_mask = (data['time'] >= start) & \
            (data['time'] <= end)
    vent_now.append(data[time_mask].copy())
    vent_now[-1]["Id"] = current_ID
    current_ID += 1

vent_now = pd.concat(vent_now, ignore_index=True)

In [5]:
vent

,time,temperature_bathroom_middle,humidity_bathroom_middle,temperature_bathoroom_upstairs,humidity_bathroom_upstairs,temperature_outside,humidity_outside,Id,Config,local time,room,absolute,absolute_outside
0,2022-01-13 06:40:00,20.4,62.0,24.1,40.0,0.0,96.0,39,1,0.0,up,1.224345,0.58656
1,2022-01-13 06:45:00,20.4,62.0,24.1,40.0,0.0,96.0,39,1,5.0,up,1.224345,0.58656
2,2022-01-13 06:50:00,20.4,64.0,24.1,40.0,0.0,96.0,39,1,10.0,up,1.224345,0.58656
3,2022-01-13 06:55:00,20.7,64.0,24.1,40.0,0.0,96.0,39,1,15.0,up,1.224345,0.58656
4,2022-01-13 07:00:00,20.7,64.0,24.1,40.0,0.0,96.0,39,1,20.0,up,1.224345,0.58656
...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,2022-01-26 09:55:00,16.3,55.0,22.0,51.0,NaN,NaN,100,4,30.0,mid,NaN,NaN
856,2022-01-26 10:00:00,16.3,55.0,22.0,51.0,NaN,NaN,100,4,35.0,mid,NaN,NaN
857,2022-01-26 10:05:00,16.3,55.0,22.4,51.0,NaN,NaN,100,4,40.0,mid,NaN,NaN
858,2022-01-26 10:10:00,16.3,55.0,22.4,51.0,NaN,NaN,100,4,45.0,mid,NaN,NaN


In [6]:
#showering = vent_now
id = shower_max
local_time = []
for i in range(len(showering)):  
    if showering.iloc[i]["Id"] != id:
        id = showering.iloc[i]["Id"]
        first_time = showering.iloc[i]["time"]
        time = 0
    else:
        time = (showering.iloc[i]["time"] - first_time).seconds//60
        
    local_time.append(time)
showering["local time"] = local_time
showering["room"] = "up"

In [7]:
vent_now.plot.scatter(x="local time", y="humidity_bathroom_middle")

In [8]:
pd.concat([showering, dush], ignore_index=True).to_csv("dush.csv", sep=";")